# Clustering and Segmentation of Credit Card Data

### Problem Description

This case requires to develop a customer segmentation to define marketing strategy for an Organization. The sample Dataset summarizes the usage behavior of about 9000 active credit card holders during the last 6 months. The file is at a customer level with 18 behavioral variables.

The Data has been collected from a Kaggle Competition [Link to Dataset: Kaggle](https://www.kaggle.com/arjunbhasin2013/ccdata)

__Author: Billy Otieno__

In [1]:
# Import all the Required Libraries
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
%matplotlib inline


# Code to reduce notebook warnings
import warnings
warnings.filterwarnings('ignore')

print("Libraries Imported!")

Libraries Imported!


In [3]:
# Confirm file exists in the directory
!ls

Analyzing Credit Card Data.ipynb
Breast Cancer Predicition - Classification Algorithms.ipynb
Classifying satisfied students with training using Logistic regression.R
Clustering and Segmentation of Credit Card Data.ipynb
credit_card_data.csv
Customer Gas Geospatial Analysis.ipynb
customer_gas_survey.csv
Dr nyawate
ecommerce_data.csv
projo.csv
README.md
so_cancer_detection
tpot_breast_cancer_pipepline.py
tpot_cleaned_data.csv
wbc_cancer_data.csv


In [5]:
credit_df = pd.read_csv('credit_card_data.csv')
credit_df.head()

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0  C10001    40.900749           0.818182      95.40              0.00   
1  C10002  3202.467416           0.909091       0.00              0.00   
2  C10003  2495.148862           1.000000     773.17            773.17   
3  C10004  1666.670542           0.636364    1499.00           1499.00   
4  C10005   817.714335           1.000000      16.00             16.00   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                    95.4      0.000000             0.166667   
1                     0.0   6442.945483             0.000000   
2                     0.0      0.000000             1.000000   
3                     0.0    205.788017             0.083333   
4                     0.0      0.000000             0.083333   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                    0.000000                          0.083333   
1                    0.000000                          0.000000   
2                    1.000000                          0.000000   
3                    0.083333                          0.000000   
4                    0.083333                          0.000000   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                0.000000                 0              2        1000.0   
1                0.250000                 4              0        7000.0   
2                0.000000                 0             12        7500.0   
3                0.083333                 1              1        7500.0   
4                0.000000                 0              1        1200.0   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0   201.802084        139.509787          0.000000      12  
1  4103.032597       1072.340217          0.222222      12  
2   622.066742        627.284787          0.000000      12  
3     0.000000               NaN          0.000000      12  
4   678.334763        244.791237          0.000000      12

In [7]:
# Check the Columns
credit_df.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

In [9]:
# Check for null values
credit_df.isna().sum()

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64

In [11]:
credit_df.shape

(8950, 18)

In [19]:
credit_df[credit_df.CREDIT_LIMIT.isna()]

CUST_ID    BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
5203  C15349  18.400472           0.166667        0.0               0.0   

      INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
5203                     0.0    186.853063                  0.0   

      ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
5203                         0.0                               0.0   

      CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
5203                0.166667                 1              0           NaN   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
5203  9.040017         14.418723               0.0       6

In [23]:
credit_df[credit_df.MINIMUM_PAYMENTS.isna()]

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
3     C10004  1666.670542           0.636364    1499.00           1499.00   
45    C10047  2242.311686           1.000000     437.00             97.00   
47    C10049  3910.111237           1.000000       0.00              0.00   
54    C10056     6.660517           0.636364     310.00              0.00   
55    C10057  1311.995984           1.000000    1283.90           1283.90   
56    C10058  3625.218146           1.000000     313.27            313.27   
63    C10065     7.152356           0.090909     840.00            840.00   
93    C10098  1307.717841           1.000000     405.60            405.60   
94    C10099  2329.485768           1.000000     213.34            213.34   
97    C10102  3505.671311           1.000000       0.00              0.00   
99    C10104     0.000000           0.000000       0.00              0.00   
103   C10108     0.624240           0.090909     389.96              0.00   
111   C10116  3393.351236           1.000000       0.00              0.00   
121   C10126  1464.324632           1.000000    1174.39           1174.39   
129   C10135    69.405545           0.090909       0.00              0.00   
130   C10136   116.654149           0.272727     907.00            907.00   
142   C10148  5347.814547           1.000000       0.00              0.00   
146   C10152    19.354559           0.909091     689.70              0.00   
156   C10162  5837.105453           1.000000    3488.49           3262.25   
176   C10182  1872.926275           1.000000     766.61            766.61   
181   C10187     0.000000           0.000000      23.00              0.00   
183   C10189     5.465999           0.090909     196.81            173.00   
188   C10194   861.001207           1.000000     969.56            969.56   
191   C10198   189.687964           1.000000      66.14             66.14   
215   C10223     2.433254           0.181818      36.00              0.00   
235   C10244  4014.388265           1.000000    1189.52           1189.52   
240   C10249  3215.511032           0.818182    1112.20           1112.20   
277   C10288    32.124999           0.272727     380.89            380.89   
309   C10320   566.908563           1.000000     397.06            303.70   
312   C10323   515.904786           1.000000    5359.02           2399.78   
...      ...          ...                ...        ...               ...   
8483  C18713     2.433343           0.166667       0.00              0.00   
8484  C18714     0.000000           0.000000     120.82              0.00   
8491  C18721    13.084067           0.090909     350.00            350.00   
8500  C18731     0.000000           0.000000     113.00              0.00   
8504  C18735     0.021102           0.125000       0.00              0.00   
8505  C18736    11.918653           0.100000     276.79            276.79   
8519  C18750    11.748193           0.181818      80.75             80.75   
8561  C18794     3.229304           0.545455     190.20              0.00   
8565  C18798    19.125876           0.090909     275.86            171.98   
8583  C18817  1029.111236           0.909091       0.00              0.00   
8593  C18827    16.887571           0.363636    1083.00           1083.00   
8596  C18830    37.123618           0.857143      84.24              0.00   
8619  C18853     7.696813           0.090909     111.30            111.30   
8647  C18882     0.006651           0.142857       0.00              0.00   
8697  C18934    10.652530           0.181818     209.20              0.00   
8718  C18955    18.885677           0.375000      57.00             57.00   
8719  C18956   165.240633           0.181818       0.00              0.00   
8739  C18977  1367.703777           0.909091     153.00              0.00   
8743  C18982    40.960739           0.181818     472.75            431.37   
8746  C18985  2856.473263           1.000000    1200.00           1200.00   
8

#### Drop the rows with missing values since they are insignificant to the total number of columns

In [25]:
# Initial Shape
credit_df.shape

(8950, 18)

In [26]:
# Shape after dropping missing columns
credit_df.dropna(axis=0, inplace=True)
credit_df.shape

(8636, 18)

In [28]:
credit_df.dtypes

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object

In [33]:
# Check for uniqueness of the Customers ID
len(credit_df.CUST_ID.unique())

8636

In [34]:
# Remove the CustomerID, Feature since will not be effective in our clustering model
credit_df.drop(columns=['CUST_ID'], inplace=True)
credit_df.shape

(8636, 17)

In [36]:
credit_df.head()

BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0    40.900749           0.818182      95.40              0.00   
1  3202.467416           0.909091       0.00              0.00   
2  2495.148862           1.000000     773.17            773.17   
4   817.714335           1.000000      16.00             16.00   
5  1809.828751           1.000000    1333.28              0.00   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                   95.40      0.000000             0.166667   
1                    0.00   6442.945483             0.000000   
2                    0.00      0.000000             1.000000   
4                    0.00      0.000000             0.083333   
5                 1333.28      0.000000             0.666667   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                    0.000000                          0.083333   
1                    0.000000                          0.000000   
2                    1.000000                          0.000000   
4                    0.083333                          0.000000   
5                    0.000000                          0.583333   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                    0.00                 0              2        1000.0   
1                    0.25                 4              0        7000.0   
2                    0.00                 0             12        7500.0   
4                    0.00                 0              1        1200.0   
5                    0.00                 0              8        1800.0   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0   201.802084        139.509787          0.000000      12  
1  4103.032597       1072.340217          0.222222      12  
2   622.066742        627.284787          0.000000      12  
4   678.334763        244.791237          0.000000      12  
5  1400.057770       2407.246035          0.000000      12

### Normalizing our Data

In [40]:
from sklearn.preprocessing import StandardScaler

X = credit_df.values
credit_norm = StandardScaler().fit_transform(X)
credit_norm[:5]

array([[-0.74462486, -0.37004679, -0.42918384, -0.35916028, -0.35482593,
        -0.46865549, -0.82076881, -0.68627958, -0.71717894, -0.68195251,
        -0.47943688, -0.51762292, -0.96257545, -0.54394191, -0.30550763,
        -0.53772694,  0.35518066],
       [ 0.76415211,  0.06767893, -0.47320819, -0.35916028, -0.45883904,
         2.56855647, -1.2361389 , -0.68627958, -0.92652157,  0.55702219,
         0.09925796, -0.59705416,  0.67720406,  0.79685226,  0.08768873,
         0.21238001,  0.35518066],
       [ 0.42660239,  0.50540465, -0.11641251,  0.09990947, -0.45883904,
        -0.46865549,  1.25607662,  2.6466512 , -0.92652157, -0.68195251,
        -0.47943688, -0.12046673,  0.81385235, -0.39950328, -0.09990611,
        -0.53772694,  0.35518066],
       [-0.37391041,  0.50540465, -0.46582465, -0.34966028, -0.45883904,
        -0.46865549, -1.0284551 , -0.40853646, -0.92652157, -0.68195251,
        -0.47943688, -0.55733854, -0.90791614, -0.3801648 , -0.26113056,
        -0.53772694

### Modeling: Creating our Clusters

In [41]:
# Importing KMeans Clustering Model
from sklearn.cluster import KMeans 

In [42]:
number_of_clusters = 5
k_means = KMeans(init = "k-means++", n_clusters = number_of_clusters, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[0 1 1 ... 0 0 0]


In [47]:
km_cluster_centers = k_means.cluster_centers_
km_cluster_centers[:2]

array([[8.22627165e+02, 8.74327544e-01, 5.32927817e+02, 2.61304038e+02,
        2.71927472e+02, 4.92599737e+02, 4.59655176e-01, 1.36274327e-01,
        3.52382626e-01, 1.11127405e-01, 2.33109620e+00, 9.89410887e+00,
        2.25766913e+03, 9.50643847e+02, 5.42911184e+02, 1.55164053e-01,
        1.14103281e+01],
       [2.05908487e+03, 9.21175276e-01, 1.37813540e+03, 8.54528300e+02,
        5.23990123e+02, 1.06431364e+03, 5.60708720e-01, 3.17448846e-01,
        3.89403809e-01, 1.43722855e-01, 3.45986556e+00, 1.99209174e+01,
        7.35794241e+03, 1.99098742e+03, 7.85067099e+02, 1.79112696e-01,
        1.17382365e+01]])

### Evaluating the defined cluster by looking at the Silhouette Score

### Silhouette Coefficient

If the ground truth labels are not known, evaluation must be performed using the model itself. The Silhouette Coefficient **(sklearn.metrics.silhouette_score)** is an example of such an evaluation, where a higher Silhouette Coefficient score relates to a model with better defined clusters. The Silhouette Coefficient is defined for each sample and is composed of two scores:

    a: The mean distance between a sample and all other points in the same class.
    b: The mean distance between a sample and all other points in the next nearest cluster.

The Silhouette Coefficient for a set of samples is given as the mean of the Silhouette Coefficient for each sample.

**The score is bounded between -1 for incorrect clustering and +1 for highly dense clustering. Scores around zero indicate overlapping clusters.**

In [52]:
from sklearn import metrics

metrics.silhouette_score(X, labels, metric='euclidean')

0.390924087223375

### Calinski-Harabaz Index

The Calinski-Harabaz index (sklearn.metrics.calinski_harabaz_score) - also known as the Variance Ratio Criterion - can be used to evaluate the model, where a higher Calinski-Harabaz score relates to a model with better defined clusters.

In [55]:
metrics.calinski_harabaz_score(X, labels) 

2523.9597348403404

### Davies-Bouldin Index

The **Davies-Bouldin index (sklearn.metrics.davies_bouldin_score)** can be used to evaluate the model, where a lower Davies-Bouldin index relates to a model with better separation between the clusters.

In [57]:
from sklearn.metrics import davies_bouldin_score

davies_bouldin_score(X, labels)

1.1770327512012708